### A pre-trained CNN is used to identify objects within the image. CNNs are effective for image recognition tasks and can detect and classify objects in an image using inception v3

In [3]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Loading the pre-trained InceptionV3 model
model = InceptionV3(weights='imagenet')

# Loading and preprocessing the input image
img_path = 'examples/81641.jpg'
img=plt.imread(img_path)
plt.imshow(img)
plt.show()
img = image.load_img(img_path, target_size=(299, 299))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

model.summary()
# Get model predictions
predictions = model.predict(img_array)

# Decode and print the top-3 predicted classes
decoded_predictions = decode_predictions(predictions, top=3)[0]
print("Predictions:")
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({score:.2f})")


FileNotFoundError: [Errno 2] No such file or directory: 'examples/81641.jpg'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import pickle
import os

In [ ]:
images_folder = 'flickr30k/Images'
captions_file = 'flickr30k/captions.txt'
# Loading the captions from the file
with open(captions_file, 'r') as f:
    captions = f.readlines()
    
# Creating lists to store image paths and corresponding captions
image_paths = []
caption_texts = []

for caption in captions[1:]:
    image_id,caption_text= caption.strip().split('.jpg,')
    image_path = os.path.join(images_folder, image_id+'.jpg')
    image_paths.append(image_path)
    caption_texts.append(caption_text.strip('"'))

for i in range(5):
    print(image_paths[i]+'<---->'+caption_texts[i])

In [ ]:
# Tokenize captions
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(caption_texts)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Convert captions to sequences of integers
sequences = tokenizer.texts_to_sequences(caption_texts)

# Pad sequences to a fixed length
max_caption_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_caption_length, padding='post')

In [ ]:
# Load and preprocess images
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((299, 299))  # Resize image to fit InceptionV3 input size
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array

# Create a list to store image arrays
image_arrays = [load_and_preprocess_image(image_path) for image_path in image_paths]
image_arrays = np.array(image_arrays)

In [1]:
# Spliting the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(image_arrays, padded_sequences, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Saving the preprocessed data and tokenizer for future use
np.save('X_train.npy', X_train)
np.save('X_val.npy', X_val)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_val.npy', y_val)
np.save('y_test.npy', y_test)

NameError: name 'train_test_split' is not defined